Note: 데이터 로딩, 랭글링에 관한 자료정리 <br/> 
Major References:
- python for Data Analysis, O'Reilly, WesMckinney
- Getting and Cleaning Data, Coursera, John's Hopkins
---

# 1. Data Loading, Storage, and File Formats

In [10]:
import pandas as pd

##데이터 로딩

In [11]:
#!cat for printing
!cat ex-pydatabook/ch06/ex1.csv 

a,b,c,d,message
1,2,3,4,hello
5,6,7,8,world
9,10,11,12,foo

In [12]:
#Read .CSV file into a DataFrame
df = pd.read_csv('ex-pydatabook/ch06/ex1.csv')

#names: 아래와 같이 헤더(인덱스)를 지정해 줄 수 있음
df = pd.read_csv('ex-pydatabook/ch06/ex1.csv', names= ['a','b','c','d','message'])

#index_col: 컬럼 하나를 인덱싱으로 지정할 수 있음
nm = ['a','b','c','d','message']
df = pd.read_csv('ex-pydatabook/ch06/ex1.csv', names = nm , index_col='message')

#계층적 색인 (hierarchical index)
!cat ex-pydatabook/ch06/csv_mindex.csv

key1,key2,value1,value2
one,a,1,2
one,b,3,4
one,c,5,6
one,d,7,8
two,a,9,10
two,b,11,12
two,c,13,14
two,d,15,16


In [13]:
parsed = pd.read_csv('ex-pydatabook/ch06/csv_mindex.csv', index_col = ['key1','key2'])
parsed

value1  value2
key1 key2                
one  a          1       2
     b          3       4
     c          5       6
     d          7       8
two  a          9      10
     b         11      12
     c         13      14
     d         15      16

In [14]:
#csv가 아니고 txt 같은 fixed delimiter가 없는 파일 및 데이터 형식에서는 공백,',',파이선 정규표현식을 이용
#sep
list(open('ex-pydatabook/ch06/ex3.txt')) #txt는 open 으로?
#!cat ex-pydatabook/ch06/ex3.txt 와 비교했을때, 구분자를 확인하기 힘듬
result = pd.read_table('ex-pydatabook/ch06/ex3.txt',sep='\s+') # \s+ --> 공백문자 처리


####  # read_table과 read_csv의 차이점:
- read_csv: 파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어옴. 데이터 구분자는 쉼표(,)가 기본
- read_table: 파일, URL 또는 파일과 유사한 객체로부터 구분된 데이터를 읽어옴. 데이터 구분자는 탭('\t')가 기본
- 즉, 둘 다 사용을 해도 되지만 가급적 read_csv 같은 경우는 csv 파일만 사용을 하고 나머지 특별한 경우를 read_table로 활용

#### # read_table/read_csv function arguments 
파서 함수는 파일 형식에서 발생할 수 있는 매우 다양한 예외를 잘 처리할 수 있도록 많은 추가 인자를 가지고 있음 (책 참조)

**주요 function arguments:**
- skiprows
- na_values: 누락된 값이나 특정한 값들을 NaN으로 처리
- nrows: 처음 몇줄만 읽음


#### # Manually working with Delimited Formats

malformed 데이터를 다뤄야 하는 상황이 적지 않기 때문에, 매뉴얼하게 데이터를 로딩해야 하는 경우들이 생김. 'read_table' 로 다루기 힘들 데이터들

In [15]:
!cat ex-pydatabook/ch06/ex7.csv

"a","b","c"
"1","2","3"
"1","2","3","4"


(나 솔직히 이 파트 요지를 모르겠음)

In [16]:
import csv
f = open('ex-pydatabook/ch06/ex7.csv')

#csv.reader()
#이를 통해 line iterating을 할 수 있음. 아래 두줄은 한 set와 같음
reader = csv.reader(f) 
for line in reader:
    print line

['a', 'b', 'c']
['1', '2', '3']
['1', '2', '3', '4']


In [17]:
#위에 과정을 아래처럼 한번에 요약가능
lines = list(csv.reader(open('ex-pydatabook/ch06/ex7.csv')))
lines

[['a', 'b', 'c'], ['1', '2', '3'], ['1', '2', '3', '4']]

In [18]:
header,values = lines[0],lines[1:] # decomposition
data_dict = {h:v for h, v in zip(header, zip(*values))}
print data_dict

{'a': ('1', '1'), 'c': ('3', '3'), 'b': ('2', '2')}


#### #CSV Note
- 좀 더 복잡하거나 구분자가 한 글자를 초과하는 고정 길이를 가진다면 csv 모듈을 사용불가
- 이경우 줄을 나누고 문자열의 split 메서드나 정규표현식 메서드인 re.split 등을 이용해서 가공하는 작업을 해야 함

** # json 타입 처리 ** <br/>
** # XML, HTML Web Scraping (웹 내용 긁어오기) **<br/>
** # Interacting with HTML & Web APIs **<br/>
** # Interacting with Database **<br/>
상기 두 토픽관련내용은 책 참조






# 2. Combining and Merging Data Sets

** # 내장함수를 이용해 데이터 합치기 **  
- pandas.merge는 하나 이상의 키를 기준으로 DataFrame의 로우를 합침  
- SQL이나 다른 관계형 데이터 베이스의 join 연산과 유사  
- pandas.concat은 하나의 축을 따라 객체를 이어붙임  
- combine_first 인스턴스 메서드는 두 객체를 포개서 한 객체에서 누락된 데이터를 다른 객체에 있는 값으로 채우게 할 수 있음  

## DB style Dataframe Merges [Key를 이용해 row 병합]

키를 사용해서 dataframe의 row를 합침  
> 
```css
pd.merge(dfX,dfY,on='키값')
```
>

In [43]:
df1 = pd.DataFrame({'key':['b','b','a','c','a','a','b'], 'data1':range(7)})
df2 = pd.DataFrame({'key':['a','b','d'],'data2':range(3)})

In [37]:
pd.merge(df1,df2) # 특정 column을 지정하지 않으면, 겹치는 값을 column을 key로 사용

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [42]:
pd.merge(df1,df2,on='key') # 알아서 키를 선택한다고 해도, 지정하는것이 좋은 습관

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


df1에서 c와 df2에서 d는 교차 *intersection* 하지 않으므로 값 누락(교차를 기본 전제로함)

** # key 값이 겹치지 않을때: 별도 지정 ** (left_on, right_on)  
>```css
>pd.merge(dfX,dfY,left_on='~~~',right_on='~~~')
>```  

앞선 예처럼, 교차하지 않는 'c'와 'd'에 해당하는 값 누락  
merge 함수는 기본적으로 내부조인(inner join)을 수행하여 교집합인 결과를 반환


In [53]:
df3= pd.DataFrame({'lkey':['b','b','a','c','a','a','b'],'data1':range(7)})
df4= pd.DataFrame({'rkey':['a','b','d'], 'data2':range(3)})

#pd.merge(df3,df4) # 이렇게 merge하면 ERROR 발생!

pd.merge(df3,df4,left_on = 'lkey',right_on= 'rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


** # 외부 Join **  

>```css 
> pd.merge(df1,df2, how = 'outer/right/left/inner')
>```  

** function arguemnts: **
- 'left': 왼쪽 key 기준 외부 join. 
- 'right': 오른쪽 key 기준 외부 join. 
- 'outer': union of the keys. 합집합 결과 keys from both dataframes
- 'inner': 내부조인. use intersections

In [84]:
#example
df1 = pd.DataFrame({'key':['b','b','a','c','a','b'],'data1':range(6)})
df2 = pd.DataFrame({'key':['a','b','a','b','d'],'data2':range(5)})
print df1
print df2
#merge by outer joining
pd.merge(df1,df2,on='key',how='left')

   data1 key
0      0   b
1      1   b
2      2   a
3      3   c
4      4   a
5      5   b
   data2 key
0      0   a
1      1   b
2      2   a
3      3   b
4      4   d


,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,2,a,0
5,2,a,2
6,3,c,NaN
7,4,a,0
8,4,a,2
9,5,b,1


위 예시에서 볼수 있듯이, df1과 df2가 서로 가지고 있지 않은 key값은 NaN으로 처리되어 테이블이 
join함  
위와 달리, `how ='inner' `로 설정하면 앞선 예시처럼 교차하지 않는 key값은 날아감 (아래예시참조)

In [56]:
pd.merge(df1,df2,how='inner') # pd.merge(df1,df2)와 동일한 효과

,data1,key,data2
0,0,b,1
1,0,b,3
2,1,b,1
3,1,b,3
4,5,b,1
5,5,b,3
6,2,a,0
7,2,a,2
8,4,a,0
9,4,a,2


** # Merge with multiple keys 복수의 키로 병합**  
column 리스트를 전달.



In [60]:
left = pd.DataFrame({'key1':['foo','foo','bar'],
                     'key2':['one','two','one'],
                     'l_val':[1,2,3]})
right = pd.DataFrame({'key1':['foo','foo','bar','bar'],
                      'key2':['one','one','one','two'],
                      'r_val':[4,5,6,7]})

pd.merge(left,right,on=['key1','key2'],how='outer')

,key1,key2,l_val,r_val
0,foo,one,1,4
1,foo,one,1,5
2,foo,two,2,NaN
3,bar,one,3,6
4,bar,two,NaN,7


key를 하나만 지정했을때의 예제. 지정한 키 기준으로 join하고 (없는건 짤림), 겹치는 칼럼 이름처리(suffixes 파라미터 설정으로 변경가능)

In [61]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,l_val,key2_right,r_val
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


## Merging on Index [Index을 이용해 column을 병합]
기본 원리는 비슷하니 자세한 내용은 [여기](http://nbviewer.ipython.org/github/re4lfl0w/ipython/blob/master/books/python_data_analysis/ch07_Data_prepare_fixing_transform_merge.ipynb#7.1.2-색인-머지하기) 활용. 문서화하다 시간 다감!

** # Merging on Index **  
>  
```css
pd.merge(leftdf1, rightdf1, left_on='키이름', right_index=True)
```
>  

** # Join method **  
index로 합치는 경우, 색인구조가 유사 & 컬럼이 겹치지 않는 경우, 여러게 dataframe를 합칠때 Join 메소드를 쓰는게 깔끔하고 편리  
>  
```css
left2.join(right2, how='outer')
```
>  



In [64]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]], index=['a', 'c', 'e'],columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.],[13,14]], index=['b', 'c','d', 'e'],columns=['Missouri', 'Alabama'])
print left2
print right2

   Ohio  Nevada
a     1       2
c     3       4
e     5       6
   Missouri  Alabama
b         7        8
c         9       10
d        11       12
e        13       14


In [71]:
#기존 Merge를 이용했을때 사용예
pd.merge(left2,right2,how='outer',left_index=True,right_index=True) 

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


In [69]:
# in other hands, join을 이용했을때. 간편함
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1,2,NaN,NaN
b,NaN,NaN,7,8
c,3,4,9,10
d,NaN,NaN,11,12
e,5,6,13,14


다만 join은 왼쪽 우선 join한다.(performs a left join on the join keys)  
index-on-index 색인 대 색인 병합은 join으로 가능하나(앞선 링크 참조), 일반적으로 `concat` 메소드를 이용함

## Concatenating Along an Axis

axis = 0 : row 합침 (기본)  
axis = 1 : column 합침

** # 기본 명령어 ** 
>  
```css
pd.concat([s1,s2,s3], axis = 0 or axis =1)
```
>  

In [78]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

pd.concat([s1,s2,s3], axis = 0) # axis에 따라 row/column 병합 방향 달라짐

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

** # join 파라미터 설정으로 교집합 설정가능 **  
>  
```css
pd.concat([s1, s4], axis=1, join='inner')
```
>  

In [81]:
s4 = pd.concat([s1 * 5, s3])
pd.concat([s1, s4], axis=1)

,0,1
a,0,0
b,1,5
f,NaN,5
g,NaN,6


** # Merge하려는 축을 개별로 지정 가능 **  
>  
```css
pd.concat([s1, s4],axis=1,join_axes=[['a','c','b','e']])
```
>  

In [82]:
pd.concat([s1, s4],axis=1,join_axes=[['a','c','b','e']])

,0,1
a,0,0
c,NaN,NaN
b,1,5
e,NaN,NaN
